# Sihteeri - Gradio-käyttöliittymä
Tämä notebook tarjoaa Gradio-pohjaisen käyttöliittymän, jossa voit asettaa backend-URL:n, syöttää promptin, hakea käytettävissä olevat agentit sekä näyttää ja päivittää system-promptin.

In [ ]:
!pip install gradio requests --quiet

In [ ]:
import gradio as gr
import requests

def send_prompt(prompt, backend_url):
    try:
        response = requests.post(backend_url, data={"Body": prompt, "From": "gradio_user"})
        return response.text if response.status_code == 200 else f"Error: {response.status_code}"
    except requests.RequestException as e:
        return f"Request failed: {e}"

def fetch_system_prompt(backend_url):
    try:
        response = requests.post(backend_url, data={"Body": "system_prompt", "From": "gradio_user"})
        return response.text if response.status_code == 200 else f"Error: {response.status_code}"
    except requests.RequestException as e:
        return f"Request failed: {e}"

def fetch_agents(backend_url):
    try:
        response = requests.post(backend_url, data={"Body": "list_agents", "From": "gradio_user"})
        return response.text if response.status_code == 200 else f"Error: {response.status_code}"
    except requests.RequestException as e:
        return f"Request failed: {e}"

with gr.Blocks() as interface:
    with gr.Row():
        with gr.Column(scale=1):
            backend_url = gr.Textbox(label="Backend URL", value="http://localhost:5000/whatsapp")
            prompt = gr.Textbox(label="Syötä prompt")
            send_button = gr.Button("Lähetä prompt")
        
        with gr.Column(scale=1):
            agents_display = gr.Textbox(label="Käytettävissä olevat agentit", interactive=False)
            fetch_agents_button = gr.Button("Hae agentit")
            fetch_agents_button.click(fn=fetch_agents, inputs=backend_url, outputs=agents_display)
        
        with gr.Column(scale=1):
            system_prompt_display = gr.Textbox(label="System Prompt", interactive=True)
            fetch_button = gr.Button("Hae System Prompt")
            fetch_button.click(fn=fetch_system_prompt, inputs=backend_url, outputs=system_prompt_display)

    with gr.Row():
        output = gr.Textbox(label="Vastaus", interactive=False)
        send_button.click(fn=send_prompt, inputs=[prompt, backend_url], outputs=output)

interface.launch(inline=True)